# 대규모 언어 모델로 텍스트 생성하기

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl2/blob/master/10-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## EXAONE-3.5로 상품 질문에 대한 대답 생성하기

In [ ]:
from transformers import AutoTokenizer

exaone_tokenizer = AutoTokenizer.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct")

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

pipe = pipeline(task="text-generation",
                model="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
                tokenizer=exaone_tokenizer,
                device=0, trust_remote_code=True)

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
messages = [
    {"role": "system",
     "content": "너는 쇼핑몰 홈페이지에 올라온 질문에 대답하는 Q&A 챗봇이야. \
                 확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해 \
                 시간이 필요하다는 간단하고 친절한 답변을 생성해줘."},
    {"role": "user", "content": "이 다이어리에 내년도 공휴일이 표시되어 있나요?"}
]

pipe(messages, max_new_tokens=200)

[{'generated_text': [{'role': 'system',
    'content': '너는 쇼핑몰 홈페이지에 올라온 질문에 대답하는 Q&A 챗봇이야.                  확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해                  시간이 필요하다는 간단하고 친절한 답변을 생성해줘.'},
   {'role': 'user', 'content': '이 다이어리에 내년도 공휴일이 표시되어 있나요?'},
   {'role': 'assistant',
    'content': '안녕하세요! 다이어리에 내년의 공휴일이 미리 표시되어 있는지에 대해 정확한 답변을 드리기 위해서는 제품 담당자에게 확인이 필요합니다. 현재로선 직접 확인이 어려우니, 저희가 안내드릴 수 있는 방법으로는 고객센터에 연락하시거나, 제품 페이지 내의 문의 게시판을 통해 질문해 보시는 것이 좋을 것 같습니다. 담당자분께서 빠르게 답변해 주실 거예요! 감사합니다.'}]}]

In [ ]:
pipe(messages, max_new_tokens=200, return_full_text=False)

[{'generated_text': '안녕하세요! 다이어리에 내년의 공휴일이 미리 표시되어 있는지에 대해 정확한 답변을 드리기 위해서는 제품 담당자에게 확인이 필요합니다. 현재로선 직접 확인이 어려우니, 저희가 안내드릴 수 있는 방법으로는 고객센터에 연락하시거나, 제품 페이지 내의 문의 게시판을 통해 질문해 보시는 것이 좋을 것 같습니다. 담당자분께서 빠르게 답변해 주실 거예요! 감사합니다.'}]

In [ ]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년의 공휴일 정보가 포함되어 있는지에 대해 자세히 알려드리는 데 시간이 조금 걸릴 수 있습니다. 정확한 내용을 확인하려면 제조사 측이나 쇼핑몰의 고객센터에 연락하시는 것이 가장 확실할 것 같아요. 그럼 편하실 때 다시 연락주시면 도와드리겠습니다! 도움이 되셨길 바랍니다. 😊


## 토큰 디코딩 전략

### 기본 샘플링

In [ ]:
import numpy as np

logits = np.array([1, 2, 3, 4, 100])

In [ ]:
from scipy.special import softmax

probas = softmax(logits)
print(probas)

[1.01122149e-43 2.74878501e-43 7.47197234e-43 2.03109266e-42
 1.00000000e+00]


In [ ]:
np.random.multinomial(100, probas)

array([  0,   0,   0,   0, 100])

In [ ]:
probas = softmax(logits/100)
np.random.multinomial(100, probas)

array([22, 13, 10, 12, 43])

In [ ]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=10.0)
print(output[0]['generated_text'])

저는 해당Qy 관리자 입장이지않그래서요qing 특정 브랜드 또는 제조업의 공휴일 포함사항들 자세힌 조회 권한 없으므로 공식 판매처인 판매처혹은회사 페이지의 Calendar 섹션까지 직접 검색recommension 부탁할답니다 그런 디테일 확인해봐 보아지에 확실이 정확히 될 수 밖니를 알 수도요... 거기엔 언제나 전문가, 예 쇼핑몰측이나 해당 제작 다이어리 관계자하시분들 의견들이 담겨볼 시간 필요 하셨겠다 이해 부탁드립니다 감사바겠! 😉🏰💫♖❕◫★.answer.note#DiaryContent#ChRIONlogicalEntries 정확 내용 제공 위해 담당자 요청 대기 부탁드립니다 🤕‍♀⚄ #NeedAssistantRightApprovalsFirst.handle.💰 ⁳|


In [ ]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=0.001)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년의 공휴일이 미리 표시되어 있는지에 대해 정확한 답변을 드리기 위해서는 제품 담당자에게 확인이 필요합니다. 현재로선 직접 확인이 어려우니, 저희가 안내드릴 수 있는 방법으로는 고객센터에 연락하시거나, 제품 페이지 내의 문의 게시판을 통해 질문해 보시는 것이 좋을 것 같습니다. 담당자분께서 빠르게 답변해 주실 거예요! 감사합니다.


### top-k 샘플링

In [ ]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_k=10)
print(output[0]['generated_text'])

안녕하세요! 다이어리 정보에 대해 궁금하시군요. 죄송하지만, 실시간으로 모든 디테일을 확인하기는 어렵습니다. 정확한 답변을 드리려면 제품 담당자님께 직접 문의하시는 게 가장 좋을 것 같아요. 그분께서는 다이어리의 최신 정보와 공휴일 관련 내용을 자세히 알려드릴 수 있을 거예요. 혹시 연락처나 담당자 정보가 있으시다면 함께 공유해 주시면 더 도움을 드릴 수 있을 것 같습니다! 감사합니다.


In [ ]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_k=10, temperature=10.0)
print(output[0]['generated_text'])

안녕세요. 해당 다이어리 내에 내년도 전체 날짜 범위는 표시되고 그 속에서 정확한 각 국가별로 정해지지 공휴일은 아직 정확하기 확인을 못한 상황임을 죄송히 안내 드리지만용도는 충분히 활용하기 위함이기에 공휴일 날짜는 미리 알려드리는 서비스가 아직 적용되어있지 마세요._Please check directly in the diary by the year specified inside or confirm on one-to-[Your Country/Time Period]: 우리 고객지원으로 바로 도움주시거나 공식 고객서비스 번호에서 정확함 정보 확인 부탁드리요 감사함니다.] 이 정도의 응답 형식으로 이해하였으니 도움되신길 바랍니까?! 궁금하신 건 있으세요 더 물어 보시고 필요 시 전문가님께 연락 부탁드립니다. 🤩감사히!


### top-p 샘플링

In [ ]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_p=0.9)
print(output[0]['generated_text'])

네, 맞습니다! 다이어리에 내년의 공휴일들이 표시되어 있어 날짜를 쉽게 기억하고 계획을 세울 수 있게 도와주고 있어요.  하지만 가장 정확한 정보는 올해의 해당 공휴일 일정을 확인해보시는 게 좋을 것 같아요. 혹시 특정 날짜나 지역에 대한 자세한 내용이 필요하시다면, 직접 고객 서비스팀에 문의해 보시는 것도 좋은 방법이 될 거예요. 저희가 바로 확인해 드리진 못하지만, 필요한 도움을 드릴 수 있도록 최선을 다하겠습니다!


In [ ]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=0.8, top_k=100, top_p=0.9)
print(output[0]['generated_text'])

안녕하세요! 다이어리의 공휴일 정보에 대해 궁금하시군요. 정확한 답변을 드리기 위해서는 제품 담당자가 해당 다이어리의 상세 내용을 확인해야 합니다. 아마도 내년의 공휴일 정보가 다이어리 내부에 포함되어 있을 수 있지만, 가장 확실한 답변을 위해서는 저희가 직접 확인해야 할 것 같아요. 담당자분께 문의하시면 더 빠르게 도와드리실 수 있을 것 같습니다. 감사합니다!


## GPT-4o로 상품 질문에 대한 대답 생성하기

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="OpenAI 키를 입력하세요")

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

In [ ]:
print(completion.choices[0].message.content)

안녕하세요! 제품 담당자가 정확한 정보를 확인한 후에 답변드릴 수 있도록 시간이 필요합니다. 조금만 기다려 주시면 감사하겠습니다!


In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    top_p=0.9
)
print(completion.choices[0].message.content)

안녕하세요! 해당 다이어리에 내년도 공휴일이 표시되어 있는지 확인하기 위해 제품 담당자에게 문의해보겠습니다. 조금만 기다려 주시면 감사하겠습니다!


In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=1.8
)
print(completion.choices[0].message.content)

문의해 주셔서 감사합니다! 앞으로 내년도 공휴일이 다이어리에 포함되어 있는지에 대한 esclarecimento 사항은 제품 담당자에게 문의 드리겠습니다. 조금만 기다려 주시면 정확한 정보를 준비해드리겠습니다.
